# Compute residual normalization constant

In [ ]:
import os
import sys
import yaml
import time
import numpy as np
import xarray as xr
from glob import glob

In [ ]:
# from credit.data import get_forward_data

In [ ]:
sys.path.insert(0, os.path.realpath('../libs/'))
import preprocess_utils as pu

In [ ]:
config_name = os.path.realpath('data_config_6h.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

In [ ]:
varname = 'U'
ind_level = 14

In [ ]:
pu.residual_zscore_var(conf, varname, ind_level=ind_level)

In [ ]:
varname = 'U500'
ind_level = None

In [ ]:
pu.residual_zscore_var(conf, varname, ind_level=ind_level)

## `xr.apply_ufunc(np.diff)` vs `np.diff` directly

In [ ]:
ds = list_ds_train[0]
if ind_level is not None:
    ds = ds.isel(level=ind_level)

In [ ]:
ds = ds.isel(time=slice(100))

In [ ]:
var_diff = xr.apply_ufunc(
    np.diff,
    ds[varname],
    input_core_dims=[['time']],
    output_core_dims=[['time_diff']],  # Change this to a new dimension name
    vectorize=True,
    dask='allowed',
    output_dtypes=[ds[varname].dtype]
)

ds_out = var_diff.to_dataset(name='{}_diff'.format(varname))

ds_out = ds_out.assign_coords(
    time_diff=ds_out['time_diff'])

ds_out = ds_out.transpose("time_diff", "latitude", "longitude")

In [ ]:
diff1 = np.array(ds_out['U_diff'])

In [ ]:
diff2 = np.diff(np.array(ds['U']), axis=0)

In [ ]:
np.sum(diff1 - diff2)

In [ ]:
# filenames = sorted(glob(conf['zscore'][varname]))

# year_range = conf['zscore']['years_range']
# train_years = [str(year) for year in range(year_range[0], year_range[1])]
# train_files = [file for file in filenames if any(year in file for year in train_years)]

# list_ds_train = []

# for fn in train_files:
#     list_ds_train.append(get_forward_data(fn))
    
# # ------------------------------------------------------------------------------------ #
# ds_example = list_ds_train[0][varname]
# var_shape = ds_example.shape

# N_grids = var_shape[-1] * var_shape[-2]
# mean_std_save = np.empty((2,))
# mean_std_save.fill(np.nan)

# for i_fn, ds in enumerate(list_ds_train):
#     # ===================================================================== #
#     # apply np.diff
#     var_diff = xr.apply_ufunc(
#         np.diff,
#         ds[varname],
#         input_core_dims=[['time']],
#         output_core_dims=[['time_diff']],  # Change this to a new dimension name
#         vectorize=True,
#         dask='allowed',
#         output_dtypes=[ds[varname].dtype]
#     )
    
#     ds_out = var_diff.to_dataset(name='{}_diff'.format(varname))
    
#     ds_out = ds_out.assign_coords(
#         time_diff=ds_out['time_diff'])
    
#     ds_out = ds_out.transpose("time_diff", "latitude", "longitude")
    
#     # ===================================================================== #
#     # compute the mean and std from the np.diff result
    
#     ds_subset = ds_out['{}_diff'.format(varname)]
    
#     # get mean and var for the current year
#     mean_current_yr = float(ds_subset.mean())
#     var_current_yr = float(ds_subset.var())
#     L = len(ds_subset) * N_grids
    
#     print('{} - {}'.format(mean_current_yr, var_current_yr))
        
#     if i_fn == 0:
#         # if it is the first year, pass current year to the combined 
#         mean_std_save[0] = mean_current_yr
#         mean_std_save[1] = var_current_yr
#         N_samples = L
        
#     else:
#         # https://math.stackexchange.com/questions/2971315/how-do-i-combine-standard-deviations-of-two-groups
#         mean_new = (L * mean_current_yr + N_samples * mean_std_save[0]) / (L + N_samples)
#         var_new = ((L - 1) * var_current_yr + (N_samples - 1) * mean_std_save[1]) / (L + N_samples - 1)
#         var_new_adjust = (L * N_samples * (mean_current_yr - mean_std_save[0])**2) / (L + N_samples) / (L + N_samples -1)
        
#         mean_std_save[0] = mean_new
#         mean_std_save[1] = var_new + var_new_adjust
#         N_samples = N_samples + L
        
#         print('{} - {}'.format(mean_std_save[0], mean_std_save[1]))

# save_name = conf['zscore']['save_loc'] + '{}_residual_mean_std_{}.npy'.format(conf['zscore']['prefix'], varname)
# print('Save to {}'.format(save_name))
# # np.save(save_name, mean_std_save)

# start_time = time.time()
# main()
# print("--- %s seconds ---" % (time.time() - start_time))